In [ ]:
#import libs here
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import torchvision.models as models
import numpy as np
from copy import deepcopy
from glob import glob
import os

import matplotlib.pyplot as plt
from random import randint

import time
os.chdir("/content/drive/MyDrive/Saravanan/")
from initializations import initialize, negative_model, distorted_model

In [ ]:
# get datasets
from datasets import return_dataset
[[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]] = return_dataset()
datasets = [[mnist_train, mnist_test], [fashionmnist_train, fashionmnist_test], [cifar10_train, cifar10_test]]


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device is ", device)

def get_predictions(model, inputs):
  model = model.to(device)
  inputs = inputs.to(device)

  outputs = model(inputs)
  _, preds = torch.max(outputs, 1)

  return preds


def get_features_predictions_accuracy(model, testdata, features_size, num_workers=2):
  
  #define loader
  batch_size=512  
  testloader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, 
                        shuffle=False, num_workers=num_workers, pin_memory=True)
  
  # change the last later to empty to extract features
  featuremodel = deepcopy(model)
  featuremodel.fc_backup = model.fc
  featuremodel.fc = torch.nn.Sequential()
  
  
  model = model.to(device)
  featuremodel = featuremodel.to(device)
  model.eval()
  featuremodel.eval()

  datasize = len(testdata)
  features = torch.empty(0).to(device)  
  predicts = torch.empty(0).to(device)  
  running_corrects = 0

  ind = 0
  start = time.time()
  with torch.no_grad():
    for inputs, labels in testloader:      
      inputs = inputs.to(device)
      labels = labels.to(device)

      output = featuremodel(inputs).squeeze(0).double()      
      features = torch.cat((features, output))

      batch_preds = get_predictions(model, inputs)      
      running_corrects += torch.sum(batch_preds == labels.data)
      predicts = torch.cat((predicts, batch_preds))

  accuracy = running_corrects / datasize
  print('get feature and predict run time is ', round(time.time() - start, 3))

  return features, predicts, accuracy

'done'

device is  cuda:0


'done'

In [ ]:
# save files

def get_and_save_info(inroot, outroot, featuresize=512):

  inaddresses = glob("/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=*")
  print(inaddresses)
  print(len(inaddresses))
  for inadd in inaddresses:
    namelist = inadd.split('/')
    name = namelist[-1]
    name = name.replace('.pt', '')
    print("\n\n", name, '\n' + '='*len(name) )

    model = torch.load(inadd)

    if 'fmnist' in model.traindata:
      data = fashionmnist_test
    elif ('mnist' in model.traindata) and ('fmnist' not in model.traindata):
      data = mnist_test
    elif 'cifar' in model.traindata:
      data = cifar10_test

    features, predictions, acc = get_features_predictions_accuracy(model=model, testdata=data, features_size=featuresize)
    
    print('match accuracie:', acc, max(model.test_history))

    outadd = outroot + name

    torch.save(features, outadd + '_features.pt')
    torch.save(predictions, outadd + '_predictions.pt')

'done'

'done'

In [ ]:
inputroot = "/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/"
outputroot = "/content/drive/MyDrive/Saravanan/Vggnet_features_sgd/"

get_and_save_info(inputroot, outputroot)

['/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=kaiming_uniform-on-mnist_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=negative_kaiming_uniform-on-mnist_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=distorted_kaiming_uniform-on-mnist_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=kaiming_uniform-on-fmnist_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=negative_kaiming_uniform-on-fmnist_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=distorted_kaiming_uniform-on-fmnist_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=kaiming_uniform-on-cifar_trn-20epochs-sgd.pt', '/content/drive/MyDrive/Saravanan/lost_vgg_sgd_trained_data/vgg19-init=negative_kaiming_uniform-on-cifar_trn-20epochs-sgd.pt', '/con

In [ ]:
os.chdir("/content/drive/MyDrive/Saravanan/Vggnet_features_sgd")
!ls

'vgg19-init=distorted_kaiming_normal-on-cifar_trn-20epochs-sgd_features.pt'
'vgg19-init=distorted_kaiming_normal-on-cifar_trn-20epochs-sgd_predictions.pt'
'vgg19-init=distorted_kaiming_normal-on-fmnist_trn-20epochs-sgd_features.pt'
'vgg19-init=distorted_kaiming_normal-on-fmnist_trn-20epochs-sgd_predictions.pt'
'vgg19-init=distorted_kaiming_normal-on-mnist_trn-20epochs-sgd_features.pt'
'vgg19-init=distorted_kaiming_normal-on-mnist_trn-20epochs-sgd_predictions.pt'
'vgg19-init=distorted_kaiming_uniform-on-cifar_trn-20epochs-sgd_features.pt'
'vgg19-init=distorted_kaiming_uniform-on-cifar_trn-20epochs-sgd_predictions.pt'
'vgg19-init=distorted_kaiming_uniform-on-fmnist_trn-20epochs-sgd_features.pt'
'vgg19-init=distorted_kaiming_uniform-on-fmnist_trn-20epochs-sgd_predictions.pt'
'vgg19-init=distorted_kaiming_uniform-on-mnist_trn-20epochs-sgd_features.pt'
'vgg19-init=distorted_kaiming_uniform-on-mnist_trn-20epochs-sgd_predictions.pt'
'vgg19-init=kaiming_normal-on-cifar_trn-20epochs-sgd_feature